In [1]:
%run init.ipynb

C:\Users\Administrator\Anaconda3\lib\requests\__init__.py:80: RequestsDependencyWarning: urllib3 (1.25.11) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


matchzoo version 1.0.2
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]
data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`
停用表配置成功


In [2]:
preprocessor = mz.models.aNMM.get_default_preprocessor()

In [3]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of ChineseRemoveBlack => ChineseSimplified => ChineseEmotion => IsChinese => ChineseStopRemoval => ChineseTokenizeDemo => Tokenize => Lowercase => PuncRemoval:   0%| | 0/95 [00:00<?, ?it/s]Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 2.909 seconds.
Prefix dict has been built succesfully.
Processing text_left with chain_transform of ChineseRemoveBlack => ChineseSimplified => ChineseEmotion => IsChinese => ChineseStopRemoval => ChineseTokenizeDemo => Tokenize => Lowercase => PuncRemoval: 100%|█| 95/95 [00:03<00:00, 24.49it/s]
Processing text_right with chain_transform of ChineseRemoveBlack => ChineseSimplified => ChineseEmotion => IsChinese => ChineseStopRemoval => ChineseTokenizeDemo => Tokenize => Lowercase => PuncRemoval: 100%|█| 93/93 [00:00<00:00, 109.22it/s]
Building Vocabulary from a datapack.: 100%|████████████████████████████████████| 

In [4]:
# 由于加载glove模型会导致本案例运行速度会变慢，所以不加载glove模型

In [5]:
# glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=300)
# term_index = preprocessor.context['vocab_unit'].state['term_index']
# embedding_matrix = glove_embedding.build_matrix(term_index)
# l2_norm = np.sqrt((embedding_matrix * embedding_matrix).sum(axis=1))
# embedding_matrix = embedding_matrix / l2_norm[:, np.newaxis]

In [6]:
trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=2,
    num_neg=1
)
devset = mz.dataloader.Dataset(
    data_pack=dev_pack_processed
)

In [7]:
padding_callback = mz.models.aNMM.get_default_padding_callback()

trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    stage='train',
    callback=padding_callback,
)
devloader = mz.dataloader.DataLoader(
    dataset=devset,
    stage='dev',
    callback=padding_callback,
)

In [8]:
model = mz.models.aNMM()

model.params['task'] = ranking_task

# model.params['embedding'] = embedding_matrix #这里是当加载glove等模型时取消该行注释
#设置embedding系数
model.params["embedding_output_dim"]=100
model.params["embedding_input_dim"]=preprocessor.context["embedding_input_dim"]

model.params['dropout_rate'] = 0.1

model.build()

print(model, sum(p.numel() for p in model.parameters() if p.requires_grad))

aNMM(
  (embedding): Embedding(374, 100, padding_idx=0)
  (matching): Matching()
  (hidden_layers): Sequential(
    (0): Sequential(
      (0): Linear(in_features=200, out_features=100, bias=True)
      (1): ReLU()
    )
    (1): Sequential(
      (0): Linear(in_features=100, out_features=1, bias=True)
      (1): ReLU()
    )
  )
  (q_attention): Attention(
    (linear): Linear(in_features=100, out_features=1, bias=False)
  )
  (dropout): Dropout(p=0.1, inplace=False)
  (out): Linear(in_features=1, out_features=1, bias=True)
) 57703


In [9]:
optimizer = torch.optim.Adam(model.parameters(), lr = 3e-4)

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=devloader,
    validate_interval=None,
    epochs=15
)

trainer.run()

[Iter-1 Loss-1.000]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.1613 - normalized_discounted_cumulative_gain@5(0.0): 0.1613 - mean_average_precision(0.0): 0.1613



[Iter-2 Loss-1.000]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.1613 - normalized_discounted_cumulative_gain@5(0.0): 0.1613 - mean_average_precision(0.0): 0.1613



[Iter-3 Loss-1.000]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.1613 - normalized_discounted_cumulative_gain@5(0.0): 0.1613 - mean_average_precision(0.0): 0.1613



[Iter-4 Loss-1.000]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.1613 - normalized_discounted_cumulative_gain@5(0.0): 0.1613 - mean_average_precision(0.0): 0.1613



[Iter-5 Loss-1.000]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.1613 - normalized_discounted_cumulative_gain@5(0.0): 0.1613 - mean_average_precision(0.0): 0.1613



...

[Iter-11 Loss-1.000]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.1613 - normalized_discounted_cumulative_gain@5(0.0): 0.1613 - mean_average_precision(0.0): 0.1613



[Iter-12 Loss-1.000]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.1613 - normalized_discounted_cumulative_gain@5(0.0): 0.1613 - mean_average_precision(0.0): 0.1613



[Iter-13 Loss-1.000]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.1613 - normalized_discounted_cumulative_gain@5(0.0): 0.1613 - mean_average_precision(0.0): 0.1613



[Iter-14 Loss-1.000]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.1613 - normalized_discounted_cumulative_gain@5(0.0): 0.1613 - mean_average_precision(0.0): 0.1613



[Iter-15 Loss-1.000]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.1613 - normalized_discounted_cumulative_gain@5(0.0): 0.1613 - mean_average_precision(0.0): 0.1613

Cost time: 7.518690347671509s


In [10]:
import gc
gc.collect()

1125